In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [2]:
!pip install symspellpy jamo hangul_utils
!pip install symspellpy-ko
!wget https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2018/ko/ko_50k.txt
from symspellpy import SymSpell, Verbosity
from symspellpy_ko import KoSymSpell, Verbosity
from hangul_utils import split_syllable_char, split_syllables, join_jamos
from jamo import h2j, j2hcj
import pandas as pd
import unicodedata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-12-27 01:09:12--  https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2018/ko/ko_50k.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 695053 (679K) [text/plain]
Saving to: ‘ko_50k.txt.2’

ko_50k.txt.2        100%[===================>] 678.76K  --.-KB/s    in 0.04s   

2022-12-27 01:09:12 (15.5 MB/s) - ‘ko_50k.txt.2’ saved [695053/695053]



In [10]:
import pickle

In [4]:
def to_jamos(text):
  return j2hcj(h2j(text))

# 만개의 레시피 재료 목록 자소 분리

In [28]:
with open('만개의레시피_재료_목록_오타처리용.pickle','rb') as f:
  vocab = pickle.load(f)

In [29]:
def countLetters(word):
    counter = {}
    for letter in word:
        if letter not in counter:
            counter[letter]=0
        counter[letter] +=1
    return counter

dict1 = countLetters(vocab)

In [48]:
vocab = pd.DataFrame(dict1, index=['count']).T
vocab.reset_index(inplace=True)
vocab.columns = ["term", "count"]
vocab.head()

,term,count
0,두부,5934
1,당근,13820
2,고추,2015
3,브로콜리,1212
4,새우,2084


In [49]:
vocab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20706 entries, 0 to 20705
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   term    20706 non-null  object
 1   count   20706 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 323.7+ KB


In [50]:
vocab.term = vocab.term.map(to_jamos)
vocab.to_csv("만개의레시피_재료_목록_오타처리용_decomposed.txt", sep=" ", header=None, index=None)
vocab.head()

,term,count
0,ㄷㅜㅂㅜ,5934
1,ㄷㅏㅇㄱㅡㄴ,13820
2,ㄱㅗㅊㅜ,2015
3,ㅂㅡㄹㅗㅋㅗㄹㄹㅣ,1212
4,ㅅㅐㅇㅜ,2084


In [56]:
# 저장했던 리스트 불러옴
with open('gom_typo.pkl','rb') as f:
  gom = pickle.load(f)

In [57]:
for g in gom:
  idx = g.index('있어요')
  del g[0:idx + 1]
  
  if '일은' in g :
    idx = g.index('일은')
    del g[idx :]

  elif 'ㅇㅇ' in g :
    idx = g.index('ㅇㅇ')
    del g[idx :]
  
  else:
    pass

sym_spell = SymSpell(max_dictionary_edit_distance = 3)
dictionary_path = '만개의레시피_재료_목록_오타처리용_decomposed.txt'
sym_spell.load_dictionary(dictionary_path, 0, 1)

True

In [58]:
# gom_typo_fix에 오타 교정된 재료 목록 저장
gom_typo_fix = []
for g in gom:
  g_typo_fix = []
  for term in g:
    term = split_syllables(term) 
    suggestions = sym_spell.lookup(term, Verbosity.ALL, max_edit_distance = 3)
    for sugg in suggestions :
      if sugg.distance < 2:
        typofix = join_jamos(sugg.term)
        g_typo_fix.append(typofix)
        break
      else:
        pass
  gom_typo_fix.append(g_typo_fix)


# gom_typo_fix를 다시 피클로 저장

In [60]:
#gom_typo_fix를 다시 피클로 저장
with open('gom_typo_fix.pkl','wb') as f:
    pickle.dump(gom_typo_fix,f)
